# **International Finance**
Thomas de Portzamparc - 7/12/2025 

# **Module Imports**

In [ ]:
import numpy as np 
import pandas as pd
import yfinance as yf
from scipy.stats import skew, kurtosis
import re
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# **Data Loading**

In [ ]:
dict_forward = pd.read_excel("fwd_rates.xlsx", header = 0, skiprows = [2], sheet_name = None, index_col = 0)
df_forward = pd.concat(dict_forward.values(), axis = 1)
df_forward = df_forward[1:]
dict_spot = pd.read_excel("spot_rates.xls", header = [0, 1], sheet_name = None, index_col = 0)
df_spot = pd.concat(dict_spot.values(), axis = 1)

In [ ]:
# Here we will get the dollar exchange rate and remove the other unused columns to manipulate the dataframe quicker
usd_columns_spot = [col for col in df_spot if " US" in col[0] or "US " in col [0]] 
usd_columns_fwd = [col for col in df_forward if " US" in col or "US " in col]
# A lot of XUSD spot columns are missing, we may need to retreive them buy using other currency pairs

# **Data Pre - Processing**
Here we will run some pre - treatment prior to executing strategies for both spot and forward dataframes
---

## Spot dataframe

### Computing of the XUSD spot

In [ ]:
# =========================================================
# Build metadata (Cur1 / Cur2 / Type)
# =========================================================
# Extract the currency structure and associated quote type for each column.
# Titles follow the pattern: "<CUR1> TO <CUR2> - <QUOTE TYPE>".
# We standardize the quote nature into BID, OFFER, or ER (exchange rate mid).

records = []

for (title, code) in df_spot.columns:
    # Identify base and quote currencies from the "CUR1 TO CUR2" portion
    left, right = title.split("TO")
    cur1 = left.strip().split()[0].upper()
    cur2 = right.strip().split()[0].upper()
    
    # Extract the descriptive quote label from the suffix (e.g. "BID SPOT")
    raw_nature = title.split("-")[-1].strip().upper()

    # Convert the descriptive label to a standardized quote type
    if "BID" in raw_nature:
        price_type = "BID"
    elif "OFFER" in raw_nature:
        price_type = "OFFER"
    elif "EXCHANGE" in raw_nature:
        price_type = "ER"
    else:
        price_type = "OTHER"

    records.append({
        "Title": title,
        "Code": code,
        "Cur1": cur1,
        "Cur2": cur2,
        "RawNature": raw_nature,
        "Type": price_type
    })

meta = pd.DataFrame(records, index=df_spot.columns)

# Ensure consistent MultiIndex formatting across df_spot and metadata
df_spot.columns = pd.MultiIndex.from_tuples([(str(a), str(b)) for a, b in df_spot.columns])
meta.index       = pd.MultiIndex.from_tuples([(str(a), str(b)) for a, b in meta.index])


# =========================================================
# Helpers
# =========================================================

def get_leg(cur1, cur2, typ):
    """
    Select the time series corresponding to a specific currency pair (cur1 → cur2)
    and quote type (BID, OFFER, ER). Returns None if the requested leg is missing.
    """
    mask = (meta["Cur1"] == cur1) & (meta["Cur2"] == cur2) & (meta["Type"] == typ)
    idx = meta.index[mask]
    if len(idx) == 0:
        return None
    return df_spot[idx[0]]


def invert_bid_ask(bid, ask):
    """
    Convert an X/USD bid-ask pair into the corresponding USD/X pair.
    The inverted bid equals 1/ask and the inverted ask equals 1/bid.
    """
    return 1/ask, 1/bid

# Some quotes are missing we will thus input the missing values using the quote present in the dataframe 


def complete_quotes(bid, offer, mid):
    if bid is None and offer is None and mid is None:
        return None, None, None

    # Convert to series
    b = bid.copy() if bid is not None else None
    o = offer.copy() if offer is not None else None
    m = mid.copy() if mid is not None else None

    # Ensure all exist
    if b is None:
        b = pd.Series(index=o.index if o is not None else m.index, dtype=float)
    if o is None:
        o = pd.Series(index=b.index if b is not None else m.index, dtype=float)
    if m is None:
        m = pd.Series(index=b.index if b is not None else o.index, dtype=float)

    # 1. Fill mid when possible
    mask_mid = m.isna() & b.notna() & o.notna()
    m.loc[mask_mid] = (b.loc[mask_mid] + o.loc[mask_mid]) / 2

    # 2. Fill bid when possible
    mask_bid = b.isna() & m.notna() & o.notna()
    b.loc[mask_bid] = 2 * m.loc[mask_bid] - o.loc[mask_bid]

    # 3. Fill offer when possible
    mask_offer = o.isna() & m.notna() & b.notna()
    o.loc[mask_offer] = 2 * m.loc[mask_offer] - b.loc[mask_offer]

    # 4. Last resort: if both bid & offer missing but mid exists
    mask_both = b.isna() & o.isna() & m.notna()
    b.loc[mask_both] = m.loc[mask_both]
    o.loc[mask_both] = m.loc[mask_both]

    return b, o, m


# =========================================================
# Build USDX = X per USD
# =========================================================
# Construct quotes expressing the value of 1 USD in each foreign currency.
# Direct USD→X quotes are used when available; otherwise, cross rates are
# reconstructed using pivot currencies.

usdx = pd.DataFrame(index=df_spot.index)

# List all currencies appearing in Cur1 or Cur2, excluding USD
currencies = set(meta["Cur1"].unique()) | set(meta["Cur2"].unique())
currencies.discard("US")

# Start with direct USD→UK and USD→EURO pairs
for tgt in ["UK", "EURO"]:
    bid_X_USD   = get_leg("US", tgt, "BID")
    offer_X_USD = get_leg("US", tgt, "OFFER")
    mid_X_USD   = get_leg("US", tgt, "ER")

    # Fill missing values if necessary
    bid_X_USD, offer_X_USD, mid_X_USD = complete_quotes(bid_X_USD, offer_X_USD, mid_X_USD)

    if bid_X_USD is not None and offer_X_USD is not None:
        # Convert X/USD quotes into USD→X using bid/ask inversion
        bid_USD_X, offer_USD_X = invert_bid_ask(bid_X_USD, offer_X_USD)
        mid_USD_X = 1/mid_X_USD

        usdx[f"{tgt}_BID"]   = bid_USD_X
        usdx[f"{tgt}_OFFER"] = offer_USD_X
        usdx[f"{tgt}_ER"]    = mid_USD_X


# Compute cross USD→X rates via pivot currencies when no direct quote exists
pivots = ["UK", "EURO"]

for cur in sorted(currencies):
    if cur in ["UK", "EURO"]:
        continue

    for pivot in pivots:

        # Retrieve X→pivot legs
        bid_X_P   = get_leg(cur,  pivot, "BID")
        offer_X_P = get_leg(cur,  pivot, "OFFER")
        mid_X_P   = get_leg(cur,  pivot, "ER")

        # Retrieve USD→pivot legs
        bid_US_P   = get_leg("US", pivot, "BID")
        offer_US_P = get_leg("US", pivot, "OFFER")
        mid_US_P   = get_leg("US", pivot, "ER")

        # Complete missing values before using them
        bid_X_P, offer_X_P, mid_X_P = complete_quotes(bid_X_P, offer_X_P, mid_X_P)
        bid_US_P, offer_US_P, mid_US_P = complete_quotes(bid_US_P, offer_US_P, mid_US_P)
        # Skip if still incomplete (very unlikely after correction)
        if bid_X_P is None or offer_X_P is None or bid_US_P is None or offer_US_P is None:
            continue

        # Compute X per USD via the pivot:
        # Sequence: USD→pivot (using ask), then pivot→X (using bid)
        bid_USD_X   = bid_X_P   / offer_US_P
        offer_USD_X = offer_X_P / bid_US_P
        mid_USD_X   = mid_X_P / mid_US_P

        usdx[f"{cur}_BID"]   = bid_USD_X
        usdx[f"{cur}_OFFER"] = offer_USD_X
        usdx[f"{cur}_ER"]    = mid_USD_X

        break


# =========================================================
# Convert USDX → XUSD (final quoting convention)
# =========================================================
# Invert each USD→X quote to obtain the final XUSD convention (USD per unit of X).
# Bid and ask are swapped upon inversion.

xusd_df = pd.DataFrame(index=usdx.index)

for col in usdx.columns:
    cur, typ = col.split("_")

    if typ == "BID":
        xusd_df[f"{cur}_OFFER"] = 1 / usdx[col]

    elif typ == "OFFER":
        xusd_df[f"{cur}_BID"]   = 1 / usdx[col]

    else:  # ER (mid)
        xusd_df[f"{cur}_ER"]    = 1 / usdx[col]

xusd_df = xusd_df.sort_index(axis=1)
xusd_df.head(5)



### Coherence of the spot obtained 
The first thing to do here is to verify the coherence of our computing, to do this we have several ressources, chatgpt and other AI tool may help us quickly review our code but to check the coherence of our data we can look at some spots on Yfinance or do it empirically as we've done below 

In [ ]:
# Dictionnary containing the index of the comparison dataframe and the yfinance ticker to extract market data
fx_map = {
    "UK": "GBPUSD=X",
    "EURO": "EURUSD=X",
    "PHILIPPINE": "PHPUSD=X",
    "CANADIAN": "CADUSD=X",
    "NORWEGIAN": "NOKUSD=X",
    "NEW": "NZDUSD=X",
    "CZECH": "CZKUSD=X",
    "HUNGARIAN": "HUFUSD=X",
    "POLISH": "PLNUSD=X",
    "SINGAPORE": "SGDUSD=X",
    "RUSSIAN": "RUBUSD=X",
    "INDIAN": "INRUSD=X",
    "SOUTH": "ZARUSD=X",
    "INDONESIAN": "IDRUSD=X",
    "BULGARIAN": "BGNUSD=X",
    "ISRAELI": "ILSUSD=X",
    "JAPANESE": "JPYUSD=X",
    "BRAZILIAN": "BRLUSD=X",
    "SWEDISH": "SEKUSD=X",
    "THAI": "THBUSD=X",
    "AUSTRALIAN": "AUDUSD=X",
    "SWISS": "CHFUSD=X",
    "MEXICAN": "MXNUSD=X",
    "CHILEAN": "CLPUSD=X",
}

target_date = "2024-10-23"
results = {}

for name, ticker in fx_map.items():
    try:
        data = yf.download(
            ticker,
            start="2024-10-23",
            end="2024-10-24",
            progress=False,
            auto_adjust=False
        )
        
        # If no data → record NaN
        if data.empty:
            results[name+"_ER"] = float("nan")
            continue
        
        # Look for the exact date
        date_match = data.loc[data.index.strftime("%Y-%m-%d") == target_date]
        
        if len(date_match) == 0:
            results[name+"_ER"] = float("nan")
        else:
            results[name+"_ER"] = date_match["Close"].iloc[0]
    
    except Exception:
        results[name] = float("nan")

clean_results = {k: float(v.iloc[0])for k, v in results.items()}
df_check = pd.DataFrame.from_dict(clean_results,orient="index", columns=["USD per X"]) # dataframe 


# DATA comparison 
row_model = xusd_df.loc[target_date]
row_model.name = "USD_per_X_professor_data"
df_model = row_model.to_frame(name="USD_per_X_model")
comparison = df_model.join(df_check, how="inner")
comparison["abs_diff"] = comparison["USD_per_X_model"] - comparison["USD per X"]
comparison["rel_diff(%)"] = comparison["abs_diff"] / comparison["USD per X"] * 100

print(comparison.sort_values("rel_diff(%)").head(10))


Once the verification is done, we can pursue our calculus without worrying about wether our currency pairs are quoted in the wrong direction. First and foremost we will thus start by computing some log returns -> we pick this because it has the nice property that the returns are additive and because the subject encourage us to go this way 

## Forward dataframe

In [ ]:
# ============================================================
# Utility: ensure all columns are 1D numeric arrays
# ============================================================

def force_1d(df):
    """
    Ensures all columns are flat 1D arrays and numeric.
    Handles nested arrays, 2D structures, and string-based numerics.
    """
    df2 = df.copy()
    df2 = df2.sort_index()
    for c in df2.columns:
        col = df2[c]

        # Flatten nested arrays if needed
        if any(isinstance(x, np.ndarray) for x in col):
            df2[c] = col.apply(lambda x: x.flatten()[0] if isinstance(x, np.ndarray) else x)

        arr = df2[c].to_numpy()
        if hasattr(arr, "ndim") and arr.ndim == 2:
            df2[c] = arr[:, 0]

        # Convert to numeric where possible
        try:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")
        except Exception:
            pass

    return df2


# ============================================================
# Currency normalization
# ============================================================

CURRENCY_MAP = {
    "AUD":"AUSTRALIAN","AUSTRALIAN":"AUSTRALIAN",
    "EUR":"EURO","EURO":"EURO",
    "GBP":"UK","UK":"UK",
    "NZD":"NEW","NEW":"NEW",

    "BRL":"BRAZILIAN","BRAZILIAN":"BRAZILIAN",
    "CAD":"CANADIAN","CANADIAN":"CANADIAN",
    "HRK":"CROATIAN","CROATIAN":"CROATIAN",
    "CZK":"CZECH","CZECH":"CZECH",
    "HUF":"HUNGARIAN","HUNGARIAN":"HUNGARIAN",
    "INR":"INDIAN","INDIAN":"INDIAN",
    "IDR":"INDONESIAN","INDONESIAN":"INDONESIAN",
    "ILS":"ISRAELI","ISRAELI":"ISRAELI",
    "JPY":"JAPANESE","JAPANESE":"JAPANESE",
    "MXN":"MEXICAN","MEXICAN":"MEXICAN",
    "NOK":"NORWEGIAN","NORWEGIAN":"NORWEGIAN",
    "PHP":"PHILIPPINE","PHILIPPINE":"PHILIPPINE",
    "PLN":"POLISH","POLISH":"POLISH",
    "RUB":"RUSSIAN","RUSSIAN":"RUSSIAN",
    "SGD":"SINGAPORE","SINGAPORE":"SINGAPORE",
    "ZAR":"SOUTH","SOUTH":"SOUTH",
    "SEK":"SWEDISH","SWEDISH":"SWEDISH",
    "CHF":"SWISS","SWISS":"SWISS",
    "THB":"THAI","THAI":"THAI",

    "BULG":"BULGARIAN","BULGARIAN":"BULGARIAN",
    "CHILEAN":"CHILEAN"
}

def clean_cur(raw):
    """
    Normalizes currency labels and symbols into canonical identifiers.
    """
    raw = re.sub(r"[^A-Z]", "", raw.upper())
    return CURRENCY_MAP.get(raw, raw)


# ============================================================
# Metadata extractors
# ============================================================

def extract_source(title):
    """
    Extracts the vendor/source tag from a column label.
    Example: "(WMR)" → "WMR".
    """
    parts = re.findall(r"\((.*?)\)", str(title).upper())
    return parts[-1].strip() if parts else "UNK"


def classify_type(title):
    """
    Identifies whether a quote is BID, OFFER, or ER (mid).
    Robust against vendor naming inconsistencies.
    """
    t = title.upper()
    if "EXCH" in t or "EXCHANGE RATE" in t:
        return "ER"
    if "BID" in t:
        return "BID"
    if "OFFER" in t or "OFFERED" in t:
        return "OFFER"
    return "ER"


def unique_name(base, container):
    """
    Ensures unique column names after merging multiple sources.
    """
    if base not in container:
        return base
    i = 2
    while f"{base}_{i}" in container:
        i += 1
    return f"{base}_{i}"


# ============================================================
# Forward curve construction with pivot logic (UK / EURO)
# ============================================================

def build_forward_clean(df):
    """
    Cleans raw forward FX data and produces standardized USD-denominated
    forward curves for all currencies. Handles direct USD pairs, synthetic
    X/USD via a pivot (EUR or GBP), and reconstructs bid/offer/mid spreads.
    """

    df = df.copy()
    df.index = pd.to_datetime(df.index).normalize()

    # -------------------------------
    # 1. Parse raw columns
    # -------------------------------

    records = []

    for col in df.columns:
        t = col.upper()
        if "TO" not in t:
            continue

        left, right = t.split("TO", 1)
        cur1 = clean_cur(left.split()[0])
        cur2 = clean_cur(right.split()[0])

        src = extract_source(col)
        typ = classify_type(col)

        if "1M" in t:
            tenor = "1M"
        elif "1W" in t or "SW" in t:
            tenor = "1W"
        else:
            continue

        arr = df[col].to_numpy()
        if arr.ndim > 1:
            arr = arr[:, 0]

        records.append({
            "cur1": cur1,
            "cur2": cur2,
            "tenor": tenor,
            "typ": typ,
            "src": src,
            "values": arr
        })

    # -------------------------------
    # 2. Separate USD and cross pairs
    # -------------------------------

    direct = {"1M": [], "1W": []}
    cross  = {"1M": [], "1W": []}

    for r in records:
        if "USD" in (r["cur1"], r["cur2"]) or "US" in (r["cur1"], r["cur2"]):
            direct[r["tenor"]].append(r)
        else:
            cross[r["tenor"]].append(r)

    out_1M, out_1W = {}, {}

    def add(container, cur, typ, src, v):
        base = f"{cur}_{typ}_{src}"
        container[unique_name(base, container)] = v

    # -------------------------------
    # 3. Normalize direct USD pairs
    # -------------------------------

    for tenor, container in [("1M", out_1M), ("1W", out_1W)]:
        for r in direct[tenor]:
            c1, c2 = r["cur1"], r["cur2"]
            typ, src, v = r["typ"], r["src"], r["values"]

            # Case USD → X
            if c1 in ("USD","US") and c2 not in ("USD","US"):
                add(container, c2, typ, src, v)

            # Case X → USD (invert)
            elif c2 in ("USD","US") and c1 not in ("USD","US"):
                if typ == "BID":
                    # bid(X→USD) becomes OFFER(USD→X)
                             # inversion of numeric quote
                    typ = "OFFER"         # flip side  
                elif typ == "OFFER":
                    # offer(X→USD) becomes BID(USD→X)
                    typ = "BID"
                inv = 1.0 / v
                add(container, c1, typ, src, inv)


    # -------------------------------
    # 4. Extract pivot→USD (mid, bid, offer)
    # -------------------------------

    pivots = {"UK", "EURO"}
    pivot_USD = {"1M": {}, "1W": {}}

    for tenor in ["1M", "1W"]:
        tmp = {}

        for r in direct[tenor]:
            c1, c2, typ, src, v = r["cur1"], r["cur2"], r["typ"], r["src"], r["values"]

            pivot = None
            val = None

            # USD → P
            if c1 in ("USD","US") and c2 in pivots:
                pivot, val = c2, v

            # P → USD
            # Case X → USD (needs full bid/offer inversion)
            elif c2 in ("USD","US") and c1 not in ("USD","US"):
                if typ == "BID":
                    # bid(X→USD) becomes OFFER(USD→X)
                             # inversion of numeric quote
                    typ = "OFFER"         # flip side  
                elif typ == "OFFER":
                    # offer(X→USD) becomes BID(USD→X)
                    typ = "BID"
                inv = 1.0 / v
                add(container, c1, typ, src, inv)


            if pivot is None:
                continue

            key = (pivot, src)
            tmp.setdefault(key, {"BID":None, "OFFER":None, "ER":None})
            tmp[key][typ] = val

        # Build mid if necessary
        for key, d in tmp.items():
            if d["ER"] is None:
                if d["BID"] is not None and d["OFFER"] is not None:
                    d["ER"] = (d["BID"] + d["OFFER"]) / 2
                else:
                    continue
            pivot_USD[tenor][key] = d

    # -------------------------------
    # 5. Synthetic X/USD via pivot
    # -------------------------------

    priority = {"WMR":1, "WM":1, "LSEG DS":2, "TR":3, "REFINITIV":4, "UNK":9}

    def best_source(keys):
        return sorted(keys, key=lambda k: priority.get(k[1], 5))[0]
    
    for tenor, container in [("1M", out_1M), ("1W", out_1W)]:
    
        for r in cross[tenor]:
            X, P, typ, src, vXP = r["cur1"], r["cur2"], r["typ"], r["src"], r["values"]
    
            # On ne construit le cross que sur la ligne mid
            if typ != "ER" or P not in pivots:
                continue
    
            # On récupère le meilleur pivot→USD (P par USD)
            candidates = {k: d for k, d in pivot_USD[tenor].items() if k[0] == P}
            if not candidates:
                continue
    
            k_best = best_source(list(candidates.keys()))
            P_info = candidates[k_best]
    
            mid_PUSD   = P_info["ER"]      # P par USD
            bid_PUSD   = P_info["BID"]     # P par USD
            offer_PUSD = P_info["OFFER"]   # P par USD
    
            # mid XUSD = (P/USD) / (P/X)
            mid_XUSD = mid_PUSD / vXP
            add(container, X, "ER", src, mid_XUSD)
    
            # on récupère aussi bid/offer de X→P dans la même source
            sib_bid, sib_offer = None, None
            for s in cross[tenor]:
                if s["cur1"] == X and s["cur2"] == P and s["src"] == src:
                    if s["typ"] == "BID":
                        sib_bid = s["values"]      # P par X
                    elif s["typ"] == "OFFER":
                        sib_offer = s["values"]   # P par X
    
            if (
                sib_bid   is not None and
                sib_offer is not None and
                bid_PUSD   is not None and
                offer_PUSD is not None
            ):
                # BID_XUSD   = bid_PUSD   / offer_XP
                # OFFER_XUSD = offer_PUSD / bid_XP
                bid_XUSD   = bid_PUSD   / sib_offer
                offer_XUSD = offer_PUSD / sib_bid
    
                add(container, X, "BID",   src, bid_XUSD)
                add(container, X, "OFFER", src, offer_XUSD)
    

    # -------------------------------
    # 6. Final dataframes
    # -------------------------------

    fwd_1M = pd.DataFrame(out_1M, index=df.index).sort_index(axis=1)
    fwd_1W = pd.DataFrame(out_1W, index=df.index).sort_index(axis=1)

    return fwd_1M, fwd_1W


# ============================================================
# Duplicate fusion and reconstruction
# ============================================================

SOURCE_PRIORITY = ["LSEG DS", "LSEG", "BBI", "TR", "WMR", "UNK"]

def source_rank(src):
    src = src.upper()
    for i, tag in enumerate(SOURCE_PRIORITY):
        if tag in src:
            return i
    return len(SOURCE_PRIORITY)


def complete_quotes(b, o, m):
    """
    Reconstructs a consistent bid/offer/mid triplet.
    Ensures:
        bid <= mid <= offer
        Deltas are minimized while preserving available values.
    """
    b = b.copy() if b is not None else None
    o = o.copy() if o is not None else None
    m = m.copy() if m is not None else None

    # Mid reconstruction
    if m is None:
        m = (b + o) / 2 if (b is not None and o is not None) else None

    # Bid reconstruction
    if b is None and o is not None and m is not None:
        b = 2*m - o

    # Offer reconstruction
    if o is None and b is not None and m is not None:
        o = 2*m - b

    # Arbitrage fix
    if b is not None and o is not None:
        mask = b > o
        if mask.any():
            mid_fix = (b + o) / 2
            b[mask] = mid_fix[mask]
            o[mask] = mid_fix[mask]

    if m is not None and b is not None:
        m = m.clip(lower=b)
    if m is not None and o is not None:
        m = m.clip(upper=o)

    return b, o, m


def fuse_duplicates(df):
    """
    Merges multiple vendor sources for the same currency forward.
    Selects best source by ranking, fills gaps, and reconstructs
    arbitrage-consistent bid/offer/mid triplets.
    """
    final = {}
    groups = {}

    # Group by currency / type
    for col in df.columns:
        if "_" not in col:
            continue
        parts = col.split("_")
        if len(parts) < 3:
            continue
        cur, typ = parts[0], parts[1]
        src = "_".join(parts[2:])
        groups.setdefault(cur, {}).setdefault(typ, []).append((col, src))

    for cur, type_dict in groups.items():

        def merged(type_name):
            if type_name not in type_dict:
                return None
            cols = sorted(type_dict[type_name], key=lambda x: source_rank(x[1]))
            out = df[cols[0][0]].copy()
            for col_name, _ in cols[1:]:
                out = out.fillna(df[col_name])
            return out

        b = merged("BID")
        o = merged("OFFER")
        m = merged("ER")

        b2, o2, m2 = complete_quotes(b, o, m)

        final[f"{cur}_BID"]   = b2
        final[f"{cur}_OFFER"] = o2
        final[f"{cur}_ER"]    = m2

    return pd.DataFrame(final, index=df.index).sort_index(axis=1)


# ============================================================
# Pipeline
# ============================================================

df_forward = force_1d(df_forward)
forward_1M, forward_1W = build_forward_clean(df_forward)

forward_1M = fuse_duplicates(forward_1M)
forward_1W = fuse_duplicates(forward_1W)
print(forward_1W.columns == forward_1M.columns)
forward_1W

### Coherence of the forward obtained 
The first thing to do here is to verify the coherence of our computing, to do this we have several ressources, chatgpt and other AI tool may help us quickly review our code but to check the coherence of our data we can look at some spots on Yfinance or do it empirically as we've done below 

In [ ]:
# Dictionnary containing the index of the comparison dataframe and the yfinance ticker to extract market data
fx_map = {
    "UK": "GBPUSD=X",
    "EURO": "EURUSD=X",
    "PHILIPPINE": "PHPUSD=X",
    "CANADIAN": "CADUSD=X",
    "NORWEGIAN": "NOKUSD=X",
    "NEW": "NZDUSD=X",
    "CZECH": "CZKUSD=X",
    "HUNGARIAN": "HUFUSD=X",
    "POLISH": "PLNUSD=X",
    "SINGAPORE": "SGDUSD=X",
    "RUSSIAN": "RUBUSD=X",
    "INDIAN": "INRUSD=X",
    "SOUTH": "ZARUSD=X",
    "INDONESIAN": "IDRUSD=X",
    "BULGARIAN": "BGNUSD=X",
    "ISRAELI": "ILSUSD=X",
    "JAPANESE": "JPYUSD=X",
    "BRAZILIAN": "BRLUSD=X",
    "SWEDISH": "SEKUSD=X",
    "THAI": "THBUSD=X",
    "AUSTRALIAN": "AUDUSD=X",
    "SWISS": "CHFUSD=X",
    "MEXICAN": "MXNUSD=X",
    "CHILEAN": "CLPUSD=X",
}

target_date = "2024-10-01"
results = {}

for name, ticker in fx_map.items():
    try:
        data = yf.download(
            ticker,
            start="2024-10-01",
            end="2024-10-02",
            progress=False,
            auto_adjust=False
        )
        
        # If no data → record NaN
        if data.empty:
            results[name+"_ER"] = float("nan")
            continue
        
        # Look for the exact date
        date_match = data.loc[data.index.strftime("%Y-%m-%d") == target_date]
        
        if len(date_match) == 0:
            results[name+"_ER"] = float("nan")
        else:
            results[name+"_ER"] = date_match["Close"].iloc[0]
    
    except Exception:
        results[name] = float("nan")

clean_results = {k: float(v.iloc[0])for k, v in results.items()}
df_check = pd.DataFrame.from_dict(clean_results,orient="index", columns=["USD per X"]) # dataframe 


# DATA comparison 
row_model = forward_1M.loc[target_date]
row_model.name = "USD_per_X_professor_data"
df_model = row_model.to_frame(name="USD_per_X_model")
comparison = df_model.join(df_check, how="inner")
comparison["abs_diff"] = comparison["USD_per_X_model"] - comparison["USD per X"]
comparison["rel_diff(%)"] = comparison["abs_diff"] / comparison["USD per X"] * 100
print("1M:")
print(comparison.sort_values("rel_diff(%)"))
print("\n")

# DATA comparison 
row_model = forward_1W.loc[target_date]
row_model.name = "USD_per_X_professor_data"
df_model = row_model.to_frame(name="USD_per_X_model")
comparison = df_model.join(df_check, how="inner")
comparison["abs_diff"] = comparison["USD_per_X_model"] - comparison["USD per X"]
comparison["rel_diff(%)"] = comparison["abs_diff"] / comparison["USD per X"] * 100
print("1W:")
print(comparison.sort_values("rel_diff(%)"))

In [ ]:
# we plot the missing values to check the coherence of the results -> It seems coherent 
forward_1M[["CHILEAN_ER", "BULGARIAN_ER", "SINGAPORE_ER", "CANADIAN_ER"]].plot()
plt.show()
forward_1W[["CHILEAN_ER", "BULGARIAN_ER"]].plot()
plt.show()

In [ ]:
def test_columns(df):
    names = sorted({col.split("_")[0] for col in df.columns})
    for name in names:
        bid = df[name + "_BID"]
        er = df[name + "_ER"]
        offer = df[name + "_OFFER"]
        
        same_bid_er = bid.isna().equals(er.isna())
        same_bid_offer = bid.isna().equals(offer.isna())
        same_offer_er = offer.isna().equals(er.isna())
        num_not_nan = bid.notna().sum()
        if same_bid_er and same_bid_offer and same_offer_er: 
            continue 
        else: 
            print(f"=== {name} ===")
            print("BID vs ER NaN pattern identical :", same_bid_er)
            print("BID vs OFFER NaN pattern identical :", same_bid_offer)
            print("ER vs OFFER NaN pattern identical :", same_offer_er)
            return None 
        if  num_not_nan == 0: 
            print(f"=== {name} ===")
            print("0 not Nan values there is maybe an issue") 
            break 
            return None 
    print("dataframe OK") 
            
test_columns(xusd_df)        
test_columns(forward_1W)
test_columns(forward_1M)

In [ ]:
def check_quotes(df):
    """
    verify bid <= er <= offer
    """
    cols = df.columns
    currencies = sorted({c.split("_")[0] for c in cols if "_BID" in c})

    results = {}

    for cur in currencies:
        bid_col = f"{cur}_BID"
        er_col  = f"{cur}_ER"
        off_col = f"{cur}_OFFER"

        if not ({bid_col, er_col, off_col} <= set(cols)):
            continue

        bid = df[bid_col]
        er  = df[er_col]
        off = df[off_col]

        # filtre les lignes sans NaN
        valid_mask = (~bid.isna()) & (~er.isna()) & (~off.isna())

        if valid_mask.sum() == 0:  
            results[cur] = {
                "valid": True,
                "violations": 0,
                "rows": []
            }
            continue

        bid_clean = bid[valid_mask]
        er_clean  = er[valid_mask]
        off_clean = off[valid_mask]

        cond1 = bid_clean <= er_clean
        cond2 = er_clean <= off_clean

        ok = cond1 & cond2
        bad_rows = bid_clean.index[~ok]

        results[cur] = {
            "valid": len(bad_rows) == 0,
            "violations": len(bad_rows),
            "rows": list(bad_rows)
        }

    return results


res = check_quotes(xusd_df)


# **Economic Rationale Behind the Fixed Portfolio Construction**

To obtain meaningful and interpretable results, currencies were not assigned randomly to portfolios. Instead, we grouped them according to **economic similarity, geographical proximity, market development, and common macro-financial drivers**. This approach is widely used in the empirical FX literature, as currencies tend to exhibit strong co-movements when they share similar economic fundamentals, exposure to global risk factors, or monetary and trade linkages.

The five portfolios therefore reflect **coherent currency blocs**:

1. **Developed Europe (EUR, GBP, CHF, SEK, NOK)**
   These currencies belong to highly integrated and liquid markets with similar monetary regimes and strong cross-correlations. They typically behave as low-volatility safe-haven or funding currencies, making them a natural benchmark group in FX momentum tests.

2. **Commodity-Linked Majors (AUD, NZD, CAD, ZAR, CLP)**
   These currencies are strongly exposed to global commodity cycles and international trade dynamics. Their returns tend to co-move with risk sentiment and global demand, which makes momentum effects more pronounced or more volatile within this group.

3. **Emerging Europe (CZK, PLN, HUF, HRK, BGN)**
   Central and Eastern European currencies share similar macroeconomic structures, EU economic linkages, and exposure to regional capital flows. Treating them as a unified block allows us to analyse momentum in a medium-volatility, partially integrated market segment.

4. **Emerging Asia (INR, IDR, PHP, THB, SGD)**
   Asian currencies are influenced by regional trade patterns, high growth rates, and varying degrees of managed exchange rate regimes. Grouping them together highlights how momentum behaves in markets where monetary authorities often intervene.

5. **Americas & High-Risk EM (BRL, MXN, RUB, ILS, JPY)**
   This portfolio includes currencies with higher idiosyncratic volatility, geopolitical risk, or structural risk premia. They often display strong directional moves and are useful for isolating momentum performance in high-risk environments.

This fixed allocation provides **three key benefits**:

* It generates **economically interpretable differences** across portfolios, allowing us to analyse whether momentum behaves differently in developed vs. emerging markets, commodity-linked vs. safe-haven currencies, or high-risk vs. low-risk regimes.
* It ensures **stable composition over time**, avoiding confusion between the effects of portfolio rebalancing and actual strategy performance.
* It aligns with **empirical FX research**, where currencies are often grouped by region or market characteristics to isolate structural return patterns.

Overall, structuring the portfolios economically rather than randomly creates a more robust and meaningful framework for analysing momentum strategies in the FX market.

In [ ]:
def get_economic_portfolios(df):
    """
    Build fixed economic FX portfolios.
    Input:
        df = any FX dataframe where columns follow the pattern CURRENCY or CURRENCY_xxx
             (ex: EURO, EURO_ER, EURO_BID ...)
    Output:
        A dictionary { "P1": df_subset, ..., "P5": df_subset }
    """

    # Define the 5 economic portfolios
    portfolios = {
        "P1": ["UK", "EURO", "SWISS", "SWEDISH", "NORWEGIAN"],             # Developed Europe
        "P2": ["AUSTRALIAN", "NEW", "CANADIAN", "SOUTH", "CHILEAN"],      # Commodity-linked majors
        "P3": ["CZECH", "POLISH", "HUNGARIAN", "CROATIAN", "BULGARIAN"],  # Emerging Europe
        "P4": ["INDIAN", "INDONESIAN", "PHILIPPINE", "THAI", "SINGAPORE"],# Emerging Asia
        "P5": ["BRAZILIAN", "MEXICAN", "RUSSIAN", "ISRAELI", "JAPANESE"]  # Americas + High-risk EM
    }

    out = {}
    for pname, currency_list in portfolios.items():
        name = pname +' : '
        # Select all columns in df that start with the currency name
        cols = []
        
        for cur in currency_list:
            matching = [c for c in df.columns if c.startswith(cur)]
            cols.extend(matching)
        
        name += "|".join(currency_list)
        # Build the portfolio dataframe (only relevant columns)
        out[pname] = df[cols].copy()

    return out

portfolios = get_economic_portfolios(xusd_df)
portfolios_forward_1W = get_economic_portfolios(forward_1W)
portfolios_forward_1M = get_economic_portfolios(forward_1M)

# **Trading strategies** 

## Momentum strategy

The momentum strategy implemented here is a direct translation of the assignment instructions into a systematic trading framework. It adheres to the classical structure of cross-sectional momentum in currency markets: a short lookback window used to generate directional signals, combined with a fixed monthly holding period to capture persistent trends. Its design is intentionally simple, yet it preserves market realism through the use of bid–ask spreads in execution and the separation of bid, mid, and offer data.

The signal relies on four-day cumulative log returns computed from mid-market spot prices, which provides a very short-term measure of relative performance across currencies. At each monthly rebalancing date, currencies are sorted by recent momentum, and the top and bottom quantiles form the long and short portfolios, respectively. The position is then held for twenty-two trading days. This structure enforces disciplined, calendar-based rebalancing and reflects the literature on short-horizon momentum in FX markets.

A key strength of the implementation is the explicit modelling of transaction costs through bid–ask spreads: long positions enter and exit via offer-to-bid pricing, whereas shorts transact via bid-to-offer. This prevents the strategy from overstating profitability and brings the backtest closer to executable returns. The code also ensures that signals are lagged by one day to avoid look-ahead bias.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis


# =====================================================
# 1. Split BID / ER / OFFER
# =====================================================

def split_bid_offer(df):
    bid_cols   = [c for c in df.columns if c.endswith("_BID")]
    er_cols    = [c for c in df.columns if c.endswith("_ER")]
    offer_cols = [c for c in df.columns if c.endswith("_OFFER")]

    BID   = df[bid_cols].copy()
    ER    = df[er_cols].copy()
    OFFER = df[offer_cols].copy()

    BID.columns   = [c.replace("_BID", "") for c in bid_cols]
    ER.columns    = [c.replace("_ER", "") for c in er_cols]
    OFFER.columns = [c.replace("_OFFER", "") for c in offer_cols]

    return BID, ER, OFFER



# =====================================================
# 2. Momentum signal + signals (common for both versions)
#    4-day lookback, 1-month holding (approx 22 jours)
# =====================================================

def compute_signals(ER, lookback=4, hold_period=22, pct=0.3):

    log_ret_spot = np.log(ER).diff()
    mom_signal = log_ret_spot.rolling(lookback).sum()

    dates = ER.index
    assets = ER.columns
    signals = pd.DataFrame(0, index=dates, columns=assets)

    # rebal tous les hold_period jours, après lookback
    rebalance_dates = dates[lookback::hold_period]

    for t in rebalance_dates:
        today = mom_signal.loc[t].dropna()
        if len(today) == 0:
            continue

        n = len(today)
        k = int(np.floor(n * pct))

        winners = today.nlargest(k).index
        losers  = today.nsmallest(k).index

        signals.loc[t, winners] = 1
        signals.loc[t, losers]  = -1

    # on tient les positions entre deux rebalancings
    signals = signals.replace(0, np.nan).ffill().fillna(0)

    return signals, mom_signal, log_ret_spot



# =====================================================
# 3. Momentum using only mid (ER)
#    Mark-to-market au mid, pas de coûts de transaction
# =====================================================

def momentum_ER_only(ER, signals):

    log_ret = np.log(ER).diff()
    # daily portfolio return = moyenne des positions (décalées) * log-returns
    r = (signals.shift(1) * log_ret).mean(axis=1)
    return r.fillna(0)



# =====================================================
# 4. Momentum using BID/OFFER execution rules
#    Mark-to-market au mid + coûts seulement aux changements de position
# =====================================================

def momentum_bidask(BID, OFFER, signals):

    # mid-prices and daily mid log-returns
    MID = (BID + OFFER) / 2
    log_mid = np.log(MID)
    daily_mid_ret = log_mid.diff()

    # base daily PnL: comme la version mid-only, mais avec MID
    base_ret = (signals.shift(1) * daily_mid_ret).mean(axis=1)

    # positions today vs yesterday
    pos_prev = signals.shift(1).fillna(0)
    pos_curr = signals

    log_bid   = np.log(BID)
    log_offer = np.log(OFFER)

    # transaction cost per asset / day selon type de trade
    # long open  : mid - offer  (on paie le spread en entrant long)
    # long close : bid - mid
    # short open : bid - mid
    # short close: mid - offer

    tc_long_open  = log_mid - log_offer
    tc_long_close = log_bid - log_mid
    tc_short_open = log_bid - log_mid
    tc_short_close= log_mid - log_offer

    # masks de changement de position
    long_open_mask   = (pos_prev == 0) & (pos_curr == 1)
    long_close_mask  = (pos_prev == 1) & (pos_curr == 0)
    short_open_mask  = (pos_prev == 0) & (pos_curr == -1)
    short_close_mask = (pos_prev == -1) & (pos_curr == 0)

    # flips: 1 -> -1 (close long + open short) et -1 -> 1 (close short + open long)
    flip_long_to_short = (pos_prev == 1) & (pos_curr == -1)
    flip_short_to_long = (pos_prev == -1) & (pos_curr == 1)

    long_close_mask  = long_close_mask  | flip_long_to_short
    short_open_mask  = short_open_mask  | flip_long_to_short
    short_close_mask = short_close_mask | flip_short_to_long
    long_open_mask   = long_open_mask   | flip_short_to_long

    # transaction cost matrix
    tc_matrix = (
        long_open_mask   * tc_long_open  +
        long_close_mask  * tc_long_close +
        short_open_mask  * tc_short_open +
        short_close_mask * tc_short_close
    )

    # nombre d'actifs tradés par jour
    traded_mask  = long_open_mask | long_close_mask | short_open_mask | short_close_mask
    traded_count = traded_mask.sum(axis=1).replace(0, np.nan)

    # coût de transaction au niveau portefeuille = moyenne sur les actifs tradés
    tc_port = tc_matrix.sum(axis=1) / traded_count
    tc_port = tc_port.fillna(0)

    # total daily return = mid-based MTM + transaction cost adjustments
    r = base_ret + tc_port

    return r.fillna(0)



# =====================================================
# 5. Evaluation
# =====================================================

def evaluate_strategy(r,freq = 'D'):

    r = r.fillna(0)

    perf = (1 + r).cumprod()
    if freq == 'D':
        div = 252 
    elif freq == 'M':
        div = 12  
    elif freq == 'W':
        div = 52
    ann_ret = (1 + r.mean())**div - 1
    ann_vol = r.std() * np.sqrt(div)
    sharpe  = ann_ret / ann_vol if ann_vol != 0 else np.nan

    running_max = perf.cummax()
    max_dd = ((perf - running_max) / running_max).min()

    return pd.Series({
        "Annualized Return": ann_ret,
        "Annualized Volatility": ann_vol,
        "Sharpe Ratio": sharpe,
        "Max Drawdown": max_dd,
        "Skewness (daily)": skew(r.dropna()),
        "Kurtosis (daily)": kurtosis(r.dropna(), fisher=False)
    })



# =====================================================
# 6. WRAPPER : run BOTH versions for each portfolio
# =====================================================

def run_portfolio_strategies(portfolios, lookback=4, hold_period=22, pct=0.3, print_data = True):

    results = {}

    for name, df_port in portfolios.items():

        BID, ER, OFFER = split_bid_offer(df_port)

        signals, momentum, log_ret_spot = compute_signals(
            ER, lookback=lookback, hold_period=hold_period, pct=pct
        )

        # Strategy A: mid-only
        ret_mid   = momentum_ER_only(ER, signals)
        stats_mid = evaluate_strategy(ret_mid, freq='D')

        # Strategy B: real execution using bid/offer with TC at rebal only
        ret_ba   = momentum_bidask(BID, OFFER, signals)
        stats_ba = evaluate_strategy(ret_ba, freq='D')
        
        results[name] = {
            "ER_only": {
                "returns": ret_mid,
                "stats": stats_mid
            },
            "BidAsk": {
                "returns": ret_ba,
                "stats": stats_ba
            },
            "signals": signals,
            "momentum": momentum,
            "log_returns": log_ret_spot
        }
        if print_data :
            print("\n==============================")
            print(f"Portfolio: {name}")
            print("=== ER-only ===")
            print(stats_mid)
            print("=== Bid/Ask ===")
            print(stats_ba)

    return results

results=run_portfolio_strategies(portfolios)

In [ ]:
# ============================================================
# 1) Plot ALL transactions of a portfolio
# ============================================================

def plot_portfolio_transactions(portfolio_data, portfolios, portfolio_name):

    df = portfolios[portfolio_name]
    BID, ER, OFFER = split_bid_offer(df)

    signals = portfolio_data[portfolio_name]["signals"]

    plt.figure(figsize=(14, 7))
    plt.suptitle(f"Transactions – {portfolio_name}", fontsize=14)

    for i, cur in enumerate(ER.columns):
        price = ER[cur]
        s = signals[cur]

        # raw buy/sell signals
        buy_idx  = s[s == 1].index
        sell_idx = s[s == -1].index

        # CRUCIAL FIX:
        # keep only dates actually present in price.index
        buy_idx  = buy_idx.intersection(price.index)
        sell_idx = sell_idx.intersection(price.index)

        plt.subplot(len(ER.columns), 1, i + 1)

        plt.plot(price.index, price, color="black", linewidth=1.2)
        plt.scatter(buy_idx,  price.loc[buy_idx],  color="green", marker="^", s=40)
        plt.scatter(sell_idx, price.loc[sell_idx], color="red", marker="v", s=40)

        plt.title(cur, fontsize=10)
        plt.grid(True)

    plt.tight_layout()
    plt.show()


# ============================================================
# 2) Compare portfolio PnL (ER-only vs Bid/Ask execution)
# ============================================================

def plot_portfolio_pnl_comparison(portfolio_data, portfolio_name):
    """
    Plots cumulative PnL for ER-only and Bid/Ask executed strategies.
    """
    ret_mid = portfolio_data[portfolio_name]["ER_only"]["returns"].fillna(0)
    ret_ba  = portfolio_data[portfolio_name]["BidAsk"]["returns"].fillna(0)

    pnl_mid = (1 + ret_mid).cumprod()
    pnl_ba  = (1 + ret_ba).cumprod()

    plt.figure(figsize=(14, 6))
    plt.plot(pnl_mid, linewidth=2, label="Mid-price strategy (ER-only)")
    plt.plot(pnl_ba, linewidth=2, label="Executed strategy (Bid/Ask)")

    plt.title(f"Cumulative Performance – {portfolio_name}")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Return")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()



# ============================================================
# 3) Inspect one specific currency inside a portfolio
# ============================================================

def plot_currency_details(portfolio_data, portfolios, portfolio_name, currency):
    """
    Plots:
      - ER price series
      - Buy/Sell signals
      - Executed transaction PnL (Bid/Ask)
    """

    df = portfolios[portfolio_name]
    BID, ER, OFFER = split_bid_offer(df)

    if currency not in ER.columns:
        print(f"{currency} not found in {portfolio_name}.")
        return

    price = ER[currency]
    sig   = portfolio_data[portfolio_name]["signals"][currency]

    # executed transaction returns
    long_ret  = np.log(BID[currency])   - np.log(OFFER[currency].shift(1))
    short_ret = np.log(OFFER[currency]) - np.log(BID[currency].shift(1))
    exec_ret  = ((sig.shift(1) == 1) * long_ret +
                 (sig.shift(1) == -1) * short_ret).fillna(0)

    pnl_exec = (1 + exec_ret).cumprod()

    # Plot
    plt.figure(figsize=(16, 8))

    plt.subplot(2, 1, 1)
    plt.plot(price, color="black", linewidth=1.5, label="ER price")

    buy_idx  = sig[sig == 1].index
    sell_idx = sig[sig == -1].index

    plt.scatter(buy_idx, price.loc[buy_idx], color="green", marker="^", s=60, label="Buy")
    plt.scatter(sell_idx, price.loc[sell_idx], color="red", marker="v", s=60, label="Sell")

    plt.title(f"{portfolio_name} – Price & Signals ({currency})")
    plt.xlabel("Date")
    plt.ylabel("ER")
    plt.grid(True)
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(pnl_exec, linewidth=2, color="blue")
    plt.title(f"Executed Transaction PnL – {currency}")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Return")
    plt.grid(True)

    plt.tight_layout()
    plt.show()



# ============================================================
# 4) Full portfolio summary (table + plots)
# ============================================================

def summarize_portfolio(portfolio_data, portfolios, portfolio_name):
    """
    Displays:
      - Portfolio composition
      - Summary statistics (ER-only vs Bid/Ask)
      - PnL comparison
      - Detailed transaction plots
    """
    print(f"\n====== Portfolio {portfolio_name} – Composition ======")
    print(portfolios[portfolio_name].columns.tolist())

    print(f"\n====== Performance Statistics – {portfolio_name} ======")
    stats_mid   = portfolio_data[portfolio_name]["ER_only"]["stats"]
    stats_bidask = portfolio_data[portfolio_name]["BidAsk"]["stats"]

    display(pd.DataFrame({
        "ER-only": stats_mid,
        "Bid/Ask Executed": stats_bidask
    }))

    plot_portfolio_pnl_comparison(portfolio_data, portfolio_name)
    plot_portfolio_transactions(portfolio_data, portfolios, portfolio_name)



# ============================================================
# 5) Plot cumulative PnL of ALL portfolios (ER-only & Bid/Ask)
# ============================================================

def plot_all_portfolios_pnl(portfolio_data, portfolios_keys=None):
    """
    Plots cumulative PnL for all portfolios (both ER-only and Bid/Ask).

    Parameters
    ----------
    portfolio_data : dict
        Output of run_portfolio_strategies()
    portfolios_keys : list or None
        List of portfolio names to plot (e.g., ["P1", "P2", ...]).
        If None, all available portfolios are plotted.
    """

    if portfolios_keys is None:
        portfolios_keys = sorted(list(portfolio_data.keys()))

    plt.figure(figsize=(16, 8))

    colors = ["blue", "green", "red", "purple", "orange"]
    linestyles = ["-", "--"]  # ER-only solid, Bid/Ask dashed

    for i, p in enumerate(portfolios_keys):
        data = portfolio_data[p]

        # retrieve returns
        ret_mid = data["ER_only"]["returns"].fillna(0)
        ret_ba  = data["BidAsk"]["returns"].fillna(0)

        pnl_mid = (1 + ret_mid).cumprod()
        pnl_ba  = (1 + ret_ba).cumprod()

        plt.plot(
            pnl_mid, color=colors[i % len(colors)], linestyle=linestyles[0],
            linewidth=1.8, label=f"{p} – Mid"
        )

        plt.plot(
            pnl_ba, color=colors[i % len(colors)], linestyle=linestyles[1],
            linewidth=1.8, label=f"{p} – Bid/Ask"
        )

    plt.title("Portfolio PnL Comparison (ER-only vs Bid/Ask Execution)")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Return")
    plt.grid(True)
    plt.legend(ncol=2)
    plt.tight_layout()
    plt.show()



In [ ]:
portfolio_data=run_portfolio_strategies(portfolios, print_data = False)
summarize_portfolio(portfolio_data, portfolios, "P5")
plot_all_portfolios_pnl(portfolio_data)

### **Performance Commentary**

Across all five economic FX portfolios, the strategy delivers **weak to negative performance**, with the **Bid/Ask-executed version consistently underperforming** the mid-only version. This pattern is fully consistent with a very short-term momentum signal applied to FX markets.

---

#### **Key Observations**

#### **1. The 4-day momentum signal is too noisy**

Short-horizon FX returns are dominated by noise and mean-reversion.
A 4-day signal has little persistence and reverses frequently, producing:

* low predictive power
* unstable exposures
* frequent flips
* high effective turnover

This explains why mid-only Sharpe ratios are close to zero or negative across portfolios.


#### **2. Transaction costs degrade performance sharply**

Bid/Ask execution turns marginal mid-only results into **significant losses**.
Emerging markets (P3–P5) and thinly traded currencies carry wider spreads, amplifying the drag.

In all portfolios, Sharpe ratios drop by **0.2 to 0.4 points** when costs are applied.

#### **3. Drawdowns and fat tails**

Max drawdowns are large relative to volatility, and kurtosis is exceptionally high in several portfolios, indicating that returns are dominated by rare, large shocks.
Short-term momentum performs poorly in such environments.


#### **Portfolio-Specific Notes**

* **P1 (Developed Europe)**: structurally mean-reverting FX; momentum signal fails.
* **P2 (Commodity FX)**: the only mildly positive mid-only performance, reflecting stronger trending behavior. Costs still erase the edge.
* **P3–P5 (EM FX)**: noisy price action + higher spreads → consistently negative results.


#### **Overall Assessment**

The strategy, as currently specified (4-day lookback, 22-day holding), is **not suited** to FX markets.
It captures little genuine momentum, and execution costs dominate the PnL.
Results match well-known empirical findings: **FX momentum only becomes robust at longer horizons (1–3 months)** and with proper volatility scaling.


In [ ]:
param_grid = [
    {"lookback": 4, "hold": 22, "pct": 0.2},
    {"lookback": 10, "hold": 22, "pct": 0.4},
]
# allow to test other setup -> we can maybe tune some values using optuna for bayesian optimisation of those parameters

subsamples = {
    "1999-2007": ("1999-01-01", "2007-12-31"), # (pre-crisis)
    "2008-2012": ("2008-01-01", "2012-12-31"), # (Financial Crisis + Euro Crisis)
    "2013-2019": ("2013-01-01", "2019-12-31"), # (QE, Low Volatility)
    "2020-2024": ("2020-01-01", "2024-12-31"), # (COVID, Inflation, QT)
}


results_sub = {}

for sub_name, (start, end) in subsamples.items():
    print(sub_name) 
    df_sub = xusd_df.loc[start:end]
    portfolios_sub = get_economic_portfolios(df_sub)
    if len(df_sub) < 200:
        continue

    for p in param_grid:
        lb = p["lookback"]
        hd = p["hold"]
        pct = p["pct"]
        portfolio_data = run_portfolio_strategies(portfolios_sub,lookback=lb, hold_period=hd, pct=pct)
        #summarize_portfolio(portfolio_data, portfolios, "P1") #change the key to visualize the portfolio you wish to comment on 
        #plot_all_portfolios_pnl(portfolio_data)

        key = f"{sub_name} | LB={lb}, HP={hd}, PCT={pct}"
        results_sub[key] = portfolio_data


### **Regime Performance Analysis of the FX Momentum Strategy**

**Across 1999–2007, 2008–2012, 2013–2019, 2020–2024**

The results reveal a clear regime dependence driven by volatility regimes, liquidity conditions, and structural characteristics of each economic portfolio.

### **1. 1999–2007 – Pre-Crisis, Low-to-Moderate Volatility**

#### Key observations

- All portfolios except P5 deliver **negative Sharpe ratios** in ER-only form.
- EM Americas (P5) is the only segment with a **positive Sharpe (~0.35)**.
- Developed FX (P1) and Commodity FX (P2) are mildly negative in ER-only form but drop sharply under Bid/Ask execution.
- Bid/Ask costs consistently **halve or destroy Sharpe** across all portfolios.

#### Interpretation

Momentum signals over short horizons struggle in stable pre-crisis FX markets where cross-currency trends are weak and mean-reversion dominates. P5 benefits from structural trends in BRL/MXN/RUB during the global EM carry boom.

#### Execution impact

Bid/Ask spreads increase drawdowns by **30–50%** and push most Sharpe ratios close to -0.5 to -0.8.
This suggests the signal is not strong enough to overcome transaction costs in this regime.

### **2. 2008–2012 – Crisis + Eurozone Turmoil (High Volatility)**

#### Key observations

- P2 (commodity FX) behaves exceptionally well in ER-only form: **Sharpe ~0.9**.
- P5 (Americas + high-beta EM) also performs strongly: **Sharpe 0.55**.
- Developed FX (P1) and Asia (P4) remain weak.
- Bid/Ask execution reduces performance but **does not destroy profitability** for the strongest portfolios (P2, P5).

#### Interpretation

Crisis regimes create **large directional dislocations** that momentum can exploit.
Commodity currencies produce robust trends driven by global deleveraging, oil, and risk cycles.

#### Execution impact

Bid/Ask erodes Sharpe but does not flip the signal negative for P2/P5, meaning the **signal strength exceeds transaction cost drag** in high-vol regimes.

### **3. 2013–2019 – QE, Compression of Risk Premia, Low Volatility**

#### Key observations

- This is the **worst regime for the strategy** across all portfolios.
- Nearly all portfolios exhibit **negative Sharpe ratios**, both ER-only and Bid/Ask.
- Momentum signals revert frequently in range-bound currency markets with limited directional risk.
- Bid/Ask execution amplifies losses significantly (Sharpe often < -0.8).

#### Interpretation

Post-crisis monetary regimes eliminated FX trends:
ultra-low rates, synchronized QE, and suppressed cross-currency volatility.
Momentum cannot extract signal when **macro dispersion collapses**.

#### Execution impact

Bid/Ask spreads consistently worsen Sharpe by **30–50%**, occasionally doubling drawdowns.
The strategy becomes **non-viable** in this environment.

### **4. 2020–2024 – COVID, Inflation Shock, Monetary Divergence**

#### Key observations

- Performance is mixed but materially better than during QE.
- P3 (EM Europe) and P5 (Americas/Asia EM) deliver **positive ER-only Sharpe**, though modest (~0.25–0.32).
- Developed markets (P1, P2) remain negative.
- EM portfolios again show greater trend persistence post-COVID.

#### Interpretation

The return of inflation, divergent central bank reactions, and volatility spikes reintroduced directional signals in EM FX, while G10 FX was dominated by policy reversals and noisy short-lived moves.

#### Execution impact

As spreads widened during COVID, Bid/Ask drag becomes large enough to erase nearly all profitability—even in the stronger EM portfolios.


### **Cross-Regime Synthesis (Most Important Part)**

#### **1. The strategy is highly regime-dependent**

Momentum succeeds in periods with:
- strong macro trends,
- high dispersion between economies,
- persistent directional moves (2008–2012, post-2020 selectively).

Fails in mean-reverting or low-vol regimes (2013–2019).

#### **2. EM portfolios (P3–P5) carry most of the alpha**

Because EM FX tends to trend structurally around risk cycles, carry unwinds, and policy shocks.

#### **3. Developed markets (P1–P2) struggle except during crises**

G10 FX is heavily mean-reverting under QE and exhibits limited trend strength.

#### **4. Bid/Ask execution systematically degrades returns**

Across all regimes:
- Sharpe reduction of **30–80%**,
- Drawdowns increase by **20–50%**,
- Returns frequently fall below zero.

Momentum signals are too weak to overcome transaction costs in most environments.

#### **5. Best performing segments per regime**

| Regime    | Best Portfolio(s) | Explanation                        |
| --------- | ----------------- | ---------------------------------- |
| 1999–2007 | P5                | EM structural trends               |
| 2008–2012 | P2, P5            | Crisis-driven directional moves    |
| 2013–2019 | None              | QE suppression of volatility       |
| 2020–2024 | P3, P5            | Macro divergence and EM volatility |

---

### **Overall Conclusion**

The strategy shows clear **economic sensitivity**:

- Works only in regimes with **persistent macro shocks** and **high dispersion**.
- EM portfolios reliably outperform G10 baskets.
- Transaction costs render the strategy **borderline untradable in low-volatility regimes** and significantly weaken it even when raw momentum is strong.


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis


# =====================================================
# 1. Split spot dataframe into BID / MID(ER) / OFFER
# =====================================================

def split_bid_offer(df):
    bid_cols   = [c for c in df.columns if c.endswith("_BID")]
    er_cols    = [c for c in df.columns if c.endswith("_ER")]
    offer_cols = [c for c in df.columns if c.endswith("_OFFER")]

    BID   = df[bid_cols].copy()
    ER    = df[er_cols].copy()
    OFFER = df[offer_cols].copy()

    BID.columns   = [c.replace("_BID", "") for c in bid_cols]
    ER.columns    = [c.replace("_ER", "") for c in er_cols]
    OFFER.columns = [c.replace("_OFFER", "") for c in offer_cols]

    return BID, ER, OFFER



# =====================================================
# 2. Harmonize forward dataframe into BID / MID / OFFER
# =====================================================

def harmonize_forward(df_forward):

    currencies = sorted(set(c.split("_")[0] for c in df_forward.columns))

    F_bid   = pd.DataFrame(index=df_forward.index)
    F_mid   = pd.DataFrame(index=df_forward.index)
    F_offer = pd.DataFrame(index=df_forward.index)

    for cur in currencies:
        if f"{cur}_BID" in df_forward.columns:
            F_bid[cur] = df_forward[f"{cur}_BID"]
        if f"{cur}_ER" in df_forward.columns:
            F_mid[cur] = df_forward[f"{cur}_ER"]
        if f"{cur}_OFFER" in df_forward.columns:
            F_offer[cur] = df_forward[f"{cur}_OFFER"]

    return F_bid, F_mid, F_offer



# =====================================================
# 3. Normalize weights (market-neutral)
# =====================================================

def clean_weights(signals):
    w = signals.copy()
    denom = w.abs().sum(axis=1)
    return (w.div(denom, axis=0)).fillna(0)



# =====================================================
# 4. PPP Strategy: log(F/S) with daily-expanded forwards
# =====================================================

def ppp_strategy(df_spot, df_forward, hold_period=22, pct=0.3):

    # -----------------------------------------
    # Split spot into BID / ER / OFFER
    # -----------------------------------------
    BID, ER, OFFER = split_bid_offer(df_spot)

    # -----------------------------------------
    # Extract raw forward curves
    # -----------------------------------------
    F_bid_raw, F_mid_raw, F_offer_raw = harmonize_forward(df_forward)

    # -----------------------------------------
    # Expand forwards to daily frequency (critical)
    # -----------------------------------------
    F_bid   = F_bid_raw.reindex(df_spot.index).ffill()
    F_mid   = F_mid_raw.reindex(df_spot.index).ffill()
    F_offer = F_offer_raw.reindex(df_spot.index).ffill()

    # -----------------------------------------
    # Common tradable universe
    # -----------------------------------------
    common = sorted(list(set(ER.columns) & set(F_mid.columns)))

    BID   = BID[common]
    ER    = ER[common]
    OFFER = OFFER[common]
    F_mid = F_mid[common]

    # -----------------------------------------
    # PPP / carry mispricing signal = log(F/S)
    # -----------------------------------------
    value_signal = np.log(F_mid) - np.log(ER)

    dates = value_signal.index
    assets = value_signal.columns

    signals = pd.DataFrame(0, index=dates, columns=assets)
    rebalance_dates = dates[::hold_period]

    for t in rebalance_dates:
        today = value_signal.loc[t].dropna()
        if today.empty:
            continue

        n = len(today)
        k = int(np.floor(n * pct))

        long_assets  = today.nlargest(k).index
        short_assets = today.nsmallest(k).index

        signals.loc[t, long_assets]  = 1
        signals.loc[t, short_assets] = -1

    # Hold between rebalancing dates
    signals = signals.replace(0, np.nan).ffill().fillna(0)

    # -----------------------------------------
    # Convert to market-neutral weights
    # -----------------------------------------
    weights = clean_weights(signals.shift(1))

    # -----------------------------------------
    # Mid-only theoretical PnL
    # -----------------------------------------
    logret = np.log(ER).diff()
    ret_mid = (weights * logret).sum(axis=1).fillna(0)

    # -----------------------------------------
    # Executed PnL with Bid-Ask Costs
    # -----------------------------------------
    MID = (BID + OFFER) / 2
    log_mid = np.log(MID)
    daily_mid_ret = log_mid.diff()

    pnl_mid = (weights * daily_mid_ret).sum(axis=1)

    pos_prev = signals.shift(1)
    pos_curr = signals

    log_bid = np.log(BID)
    log_offer = np.log(OFFER)

    long_open_cost  = log_mid - log_offer
    long_close_cost = log_bid - log_mid
    short_open_cost = log_bid - log_mid
    short_close_cost = log_mid - log_offer

    long_open  = (pos_prev == 0) & (pos_curr == 1)
    long_close = (pos_prev == 1) & (pos_curr == 0)

    short_open  = (pos_prev == 0) & (pos_curr == -1)
    short_close = (pos_prev == -1) & (pos_curr == 0)

    flip_long_to_short = (pos_prev == 1) & (pos_curr == -1)
    flip_short_to_long = (pos_prev == -1) & (pos_curr == 1)

    long_close |= flip_long_to_short
    short_open |= flip_long_to_short

    short_close |= flip_short_to_long
    long_open   |= flip_short_to_long

    tc_matrix = (
        long_open  * long_open_cost +
        long_close * long_close_cost +
        short_open * short_open_cost +
        short_close * short_close_cost
    )

    traded_mask = long_open | long_close | short_open | short_close
    trade_count = traded_mask.sum(axis=1).replace(0, np.nan)

    tc_port = (tc_matrix.sum(axis=1) / trade_count).fillna(0)

    ret_ba = pnl_mid + tc_port

    return ret_mid, ret_ba, signals, value_signal, logret



# =====================================================
# 5. Portfolio wrapper
# =====================================================

def run_ppp_portfolio_strategies(portfolios_spot, portfolios_forward, hold_period=22, pct=0.2, print_data=True):

    results = {}

    for name in portfolios_spot.keys():

        df_spot    = portfolios_spot[name]
        df_forward = portfolios_forward[name]

        ret_mid, ret_ba, signals_ppp, value_ppp, logret_ppp = ppp_strategy(
            df_spot,
            df_forward,
            hold_period=hold_period,
            pct=pct
        )

        stats_mid = evaluate_strategy(ret_mid, freq='D')
        stats_ba  = evaluate_strategy(ret_ba, freq='D')

        results[name] = {
            "ER_only": {"returns": ret_mid, "stats": stats_mid},
            "BidAsk":  {"returns": ret_ba,  "stats": stats_ba},
            "signals": signals_ppp,
            "value_signal": value_ppp,
            "log_returns": logret_ppp
        }

        if print_data:
            print("\n==============================")
            print(f"PPP Portfolio: {name}")
            print("=== PPP Mid-only ===")
            print(stats_mid)
            print("=== PPP Bid/Ask Execution ===")
            print(stats_ba)

    return results



# ================= RUN =================

print("1M")
results_ppp_1M = run_ppp_portfolio_strategies(
    portfolios_spot=portfolios,
    portfolios_forward=portfolios_forward_1M,
    hold_period=22
)

print("1W")
results_ppp_1W = run_ppp_portfolio_strategies(
    portfolios_spot=portfolios,
    portfolios_forward=portfolios_forward_1W,
    hold_period=7
)

summarize_portfolio(results_ppp_1M, portfolios, "P1")
plot_all_portfolios_pnl(results_ppp_1M)


### **1. Overall Takeaway**

The PPP signal **works only marginally at the 1-month horizon** and becomes **non-viable at the 1-week horizon**.
Forward–spot mispricing is too weak in your currency universe, and transaction costs eliminate almost all excess returns.


#### **2. PPP (1M) – Interpretation**

**Mid-only performance**

* Returns are low across most portfolios.
* Only P2 shows a meaningful Sharpe (≈0.38), suggesting some PPP/carry information in Scandinavian FX.
* Volatility remains low, consistent with a market-neutral FX strategy.

**Bid/Ask execution**

* Performance turns **negative for all portfolios except P2**.
* Sharpe ratios collapse.
* Drawdowns deepen significantly.
* High kurtosis (10–29) indicates exposure to tail events and announcement-driven jumps.

**Conclusion (1M):**
The PPP signal is present but too weak relative to spreads. In a small universe with low rate dispersion, the forward curve doesn’t provide enough systematic premium.


#### **3. PPP (1W) – Interpretation**

**Mid-only performance**

* Weak or negative returns for most portfolios.
* The signal behaves more like noise than a persistent valuation/carry factor.

**Bid/Ask execution**

* All portfolios become **strongly negative**.
* Sharpe ratios between –0.28 and –0.72.
* Drawdowns extremely large (up to –57%).
* Some portfolios show extreme kurtosis (>100), reflecting sensitivity to fast FX corrections.

**Conclusion (1W):**
Weekly PPP is **not economically meaningful** and **not tradable**. Short-horizon forward/spot mispricing is dominated by microstructure noise and transaction costs.

#### **4. Economic Interpretation**

* Your universe (GBP, EUR, CHF, SEK, NOK) has **limited forward rate dispersion** → weak carry/PPP signal.
* Forward curves at 1W or 1M do not generate strong cross-sectional spreads.
* Transaction costs fully absorb the small theoretical premium.

#### **5. Final Summary**

* **PPP 1M:** weak but detectable signal; not profitable after costs.
* **PPP 1W:** no signal; structurally untradable.
* **Main issue:** low cross-sectional rate differentials + high turnover relative to spreads.


In [ ]:
import matplotlib.pyplot as plt

def plot_ppp_execution_calendar(results_ppp, portfolios_forward, portfolio_name):
    """
    Visualise:
    - PPP rebalance dates
    - Trade execution dates (where bid/offer execution was possible)
    - Forward portfolio content
    """

    data = results_ppp[portfolio_name]
    ret_ba = data["BidAsk"]["returns"]
    signals = data["signals"]

    # Rebalance dates = where signals change (first differences)
    rebalance_dates = signals.diff().abs().sum(axis=1)
    rebalance_dates = rebalance_dates[rebalance_dates > 0].index

    # Execution dates = where ret_ba != 0
    pos_prev = signals.shift(1).fillna(0)
    pos_curr = signals
    
    trade_mask = (pos_prev != pos_curr).any(axis=1)
    exec_dates = trade_mask[trade_mask].index
    

    # Forward portfolio composition
    fwd_cols = portfolios_forward[portfolio_name].columns
    forward_assets = sorted(set(c.split("_")[0] for c in fwd_cols))

    plt.figure(figsize=(15, 4))

    plt.scatter(rebalance_dates, [1]*len(rebalance_dates),
                color="blue", s=50, label="PPP Rebalance dates")

    plt.scatter(exec_dates, [0]*len(exec_dates),
                color="red", s=30, label="Executed trades (Bid/Offer)")

    plt.yticks([0,1], ["Executed trades", "Rebalance trigger"])
    plt.grid(True, alpha=0.4)
    plt.title(f"PPP Execution Calendar – Portfolio {portfolio_name}")
    plt.xlabel("Date")
    plt.tight_layout()

    plt.legend(loc="upper left")

    print("\nForward currencies used in this portfolio:")
    print(forward_assets)

    plt.show()


plot_ppp_execution_calendar(results_ppp_1W, portfolios_forward_1W, "P5")

In [ ]:
param_grid = [
    {"pct": 0.2},
]
subsamples = {
    "1999-2007": ("1999-01-01", "2007-12-31"), # (pre-crisis)
    "2008-2012": ("2008-01-01", "2012-12-31"), # (Financial Crisis + Euro Crisis)
    "2013-2019": ("2013-01-01", "2019-12-31"), # (QE, Low Volatility)
    "2020-2024": ("2020-01-01", "2024-12-31"), # (COVID, Inflation, QT)
}


results_sub1W = {}
results_sub1M = {}
for sub_name, (start, end) in subsamples.items():
    xusd_df_sub = xusd_df.loc[start:end]
    forward_1M_sub = forward_1M.loc[start:end]
    forward_1W_sub = forward_1W.loc[start:end]


    portfolios_sub = get_economic_portfolios(xusd_df_sub)
    portfolios_forward_1W_sub = get_economic_portfolios(forward_1W_sub)
    portfolios_forward_1M_sub = get_economic_portfolios(forward_1M_sub)
    if len(df_sub) < 200:
        continue

    for p in param_grid:
        pct = p["pct"]

        results_ppp_1M = run_ppp_portfolio_strategies(
            portfolios_spot=portfolios_sub,
            portfolios_forward=portfolios_forward_1M_sub, hold_period=22, pct=pct
        )
        
        results_ppp_1W = run_ppp_portfolio_strategies(
            portfolios_spot=portfolios_sub,
            portfolios_forward=portfolios_forward_1W_sub, hold_period=7, pct=pct
        )
        summarize_portfolio(results_ppp_1W, portfolios_sub, "P5")
        summarize_portfolio(results_ppp_1M, portfolios_sub, "P5")
        plot_all_portfolios_pnl(results_ppp_1M)


        key = f"{sub_name} | HP={hd}, PCT={pct}"
        results_sub1M[key] = results_ppp_1W
        results_sub1W[key] = results_ppp_1M
        


#### **1. Stability of the PPP signal across regimes**

Across all subsamples, the PPP factor behaves **weakly and inconsistently**, with substantial regime dependence.
Only a few windows show meaningful Sharpe ratios, and most of the performance deteriorates once realistic execution costs are included.

The **economic conclusion** is clear:
PPP mispricing is slow-moving and weakly predictive, and its explanatory power collapses when markets are stressed or when currencies are heavily policy-driven.

#### **2. 1999–2007 (pre-crisis)**

* Mid-only performance is mixed. Some portfolios (P2, P4) reach respectable Sharpe ratios driven by carry-like structure in EM and commodity currencies.
* Bid/ask execution materially reduces performance but doesn’t fully eliminate it in the strongest portfolios.
* Drawdowns remain shallow, suggesting a stable macro environment where slow PPP deviations persist.

**Interpretation:**
PPP has some predictive power in a stable macro regime with differentiated interest rates and fewer shocks.


#### **3. 2008–2012 (financial crisis + euro crisis)**

* The signal weakens sharply; most portfolios produce low or negative Sharpe ratios even before transaction costs.
* Bid/ask execution amplifies losses, turning almost all portfolios untradable.
* Volatility spikes and kurtosis increases, reflecting disorderly FX repricing and regime discontinuities.

**Interpretation:**
Crisis periods destroy PPP-style arbitrage because FX is driven by liquidity, risk aversion and capital flows rather than valuation differentials.

#### **4. 2013–2019 (QE, low-vol regime)**

* PPP performance improves slightly; some portfolios (notably P5 in your EM bloc) produce stable mid-only returns.
* Execution costs, however, again absorb a material portion of the premium.
* High kurtosis shows that returns remain exposed to occasional macro jumps despite QE stability.

**Interpretation:**
PPP works better in calm markets, but the signal remains too small relative to spreads to generate robust trading returns.

#### **5. 2020–2024 (COVID, inflation, QT)**

* The signal almost collapses.
* Mid-only returns fluctuate around zero; bid/ask versions are deeply negative.
* Volatility and tail risk increase sharply (skewness and kurtosis rise), stressing the fragility of valuation-based signals during macro dislocations.

**Interpretation:**
Extreme macro shocks dominate FX pricing; PPP deviations adjust too slowly and become irrelevant for trading horizons.

#### **6. Portfolio P5 (EM + JPY bloc) – Detailed insight**

Consistently across windows:

* **Mid-only returns oscillate between weakly positive and negative.**
* **Bid/ask execution almost always turns the strategy negative.**
* High kurtosis (8–28) indicates large tail sensitivity.
* Deep drawdowns reflect violent EM and JPY adjustments during risk-off regimes.

**Conclusion for P5:**
PPP is not a reliable driver of EMFX returns and cannot overcome structural spreads or regime breaks.

#### **7. Final synthesis**

* **PPP is not a tradable standalone signal** across any macro regime in your dataset.
* When it works (e.g., pre-2007, QE years), the premium is modest and fragile.
* During crises (2008–12, 2020–24), it becomes strongly negative.
* Execution costs systematically eliminate any exploitable edge.

**Overall:**
The PPP factor is economically intuitive but empirically weak. Its timescale (multi-year convergence) is incompatible with high-frequency portfolio rotation, and transaction costs fully dominate the signal.

# Enhanced Momentum Strategy for FX Markets (Literature-Based)

Momentum strategies, originally documented by Jegadeesh & Titman (1993, 2001), have proven persistent across equities (Fama & French, 2012), global asset classes (Asness et al., 2013), and foreign exchange markets (Menkhoff et al., 2012).
However, research after the Global Financial Crisis showed that momentum is **fragile**, especially during market rebounds, and that naïve implementations can experience severe drawdowns (Daniel & Moskowitz, 2016).

To adapt the classical FX momentum strategy without over-engineering the model, we implement **three simple improvements** directly motivated by the academic literature.q

---

## **1. Volatility Scaling (Barroso & Santa-Clara, 2015)**

Momentum behaves poorly when its volatility spikes, particularly during crisis periods.
A straightforward improvement is to **reduce exposure when recent strategy volatility is high**.

**Implementation (very simple):**

```
vol_t   = rolling 60-day volatility of the strategy
scale_t = target_vol / vol_t
signal  = scale_t * raw_signal
```

**Why this matters:**
FX markets exhibit volatility clustering. This adjustment prevents the strategy from taking large positions in unstable periods (e.g., 2008, 2020), thereby reducing drawdowns and improving Sharpe ratios.

---

## **2. Market-Condition Filter (Daniel & Moskowitz, 2016)**

Momentum tends to crash in **sharp market reversals**. A full crash-robust model would be heavy, but a simple filter works well:

```
market_momentum = average 1-month return across all currencies

if market_momentum < 0:
    signals are reduced (for example by 50%)
else:
    full exposure
```

**Intuition:**
When FX markets collectively reverse after stress, momentum spreads compress violently.
This lightweight filter avoids trading aggressively in these regimes without requiring complex modelling.

---

## **3. Combine Cross-Sectional and Time-Series Momentum (Moskowitz, Ooi & Pedersen, 2012)**

Cross-sectional momentum (ranking currencies against each other) can be noisy.
Adding a **simple time-series component** stabilises signals and improves persistence.

**Implementation:**

```
ts_signal     = sign(50-day cumulative return of each currency)
final_signal  = 0.5 * cross_sectional + 0.5 * ts_signal
```

**Benefit:**
If a currency is a “relative winner” but not trending upward, the signal is moderated.
If both components agree, the position is reinforced.
This reduces false positive trades and improves robustness across regimes.

---

# **Summary**

With only three compact enhancements, the FX momentum strategy becomes:

* **More stable** (volatility scaling)
* **Less exposed to crashes** (market-condition filter)
* **More robust and persistent** (hybrid momentum signal)


In [ ]:
# =====================================================
# 1. Split BID / ER / OFFER
# =====================================================

def split_bid_offer(df):
    bid_cols   = [c for c in df.columns if c.endswith("_BID")]
    er_cols    = [c for c in df.columns if c.endswith("_ER")]
    offer_cols = [c for c in df.columns if c.endswith("_OFFER")]

    BID   = df[bid_cols].copy()
    ER    = df[er_cols].copy()
    OFFER = df[offer_cols].copy()

    BID.columns   = [c.replace("_BID", "") for c in bid_cols]
    ER.columns    = [c.replace("_ER", "") for c in er_cols]
    OFFER.columns = [c.replace("_OFFER", "") for c in offer_cols]

    return BID, ER, OFFER



# =====================================================
# 2. Basic momentum building blocks
# =====================================================

def compute_log_returns(ER):
    return np.log(ER).diff()

def compute_cross_sectional_momentum(log_ret, lookback):
    return log_ret.rolling(lookback).sum()

def compute_time_series_momentum(ER, ts_window):
    return np.sign(ER.pct_change(ts_window))

def combine_momentum(cs, ts):
    return 0.5 * cs + 0.5 * ts



# =====================================================
# 3. Market filter
# =====================================================

def compute_market_momentum(momentum):
    return momentum.mean(axis=1)

def market_regime_multiplier(market_mom, threshold):
    return 1 - 0.5 * (market_mom < threshold).astype(float)



# =====================================================
# 4. Signal construction
# =====================================================

def select_assets(momentum_vector, pct):
    momentum_vector = momentum_vector.dropna()
    if momentum_vector.empty:
        return [], []

    n = len(momentum_vector)
    k = int(np.floor(n * pct))

    winners = list(momentum_vector.nlargest(k).index)
    losers  = list(momentum_vector.nsmallest(k).index)

    return winners, losers


def build_signal_matrix(momentum, dates, assets, hold_period, pct):
    signals = pd.DataFrame(0, index=dates, columns=assets)
    rebalance_dates = dates[::hold_period]

    for t in rebalance_dates:
        winners, losers = select_assets(momentum.loc[t], pct)
        signals.loc[t, winners] = 1
        signals.loc[t, losers]  = -1

    return signals.replace(0, np.nan).ffill().fillna(0)



# =====================================================
# 5. Volatility scaling
# =====================================================

def compute_volatility_scaler(r, window=60, target_vol=0.10):
    vol = r.rolling(window).std()
    scale = (target_vol / vol).clip(upper=3)
    return scale



# =====================================================
# 6A. Mid-only execution (MTM daily)
# =====================================================

def compute_mid_execution(ER, signals):
    log_ret = np.log(ER).diff()
    return (signals.shift(1) * log_ret).mean(axis=1)



# =====================================================
# 6B. Correct bid/ask execution with monthly holding
# =====================================================

def compute_bidask_execution(BID, OFFER, ER, signals, hold_period=22):

    dates = ER.index
    assets = ER.columns

    daily_mid_ret = np.log(ER).diff()
    portfolio_ret = pd.Series(0.0, index=dates)

    # Detect true rebalance dates (signal changes)
    rebal_dates = signals.diff().abs().sum(axis=1)
    rebal_dates = rebal_dates[rebal_dates > 0].index
    rebal_dates = rebal_dates.sort_values()

    for i in range(len(rebal_dates)-1):

        t_entry = rebal_dates[i]
        t_exit  = rebal_dates[i+1]

        pos = signals.loc[t_entry]
        active = pos[pos != 0].index

        if len(active) == 0:
            continue

        # --- 1. Daily MTM using mid ---------------------
        mtm = (signals.loc[t_entry] * daily_mid_ret)[active]

        # Convert timestamps to index positions
        entry_idx = daily_mid_ret.index.get_loc(t_entry) + 1
        exit_idx  = daily_mid_ret.index.get_loc(t_exit)

        if entry_idx <= exit_idx:
            mtm_period = mtm.iloc[entry_idx : exit_idx + 1]
            portfolio_ret.iloc[entry_idx : exit_idx + 1] = mtm_period.mean(axis=1)

        # --- 2. Entry transaction cost -------------------
        entry_cost = 0
        for cur in active:
            if pos[cur] == 1:   # long
                entry_cost += np.log(OFFER.loc[t_entry, cur]) - np.log(ER.loc[t_entry, cur])
            else:               # short
                entry_cost += np.log(ER.loc[t_entry, cur]) - np.log(BID.loc[t_entry, cur])

        # --- 3. Exit transaction cost --------------------
        exit_cost = 0
        for cur in active:
            if pos[cur] == 1:
                exit_cost += np.log(ER.loc[t_exit, cur]) - np.log(BID.loc[t_exit, cur])
            else:
                exit_cost += np.log(OFFER.loc[t_exit, cur]) - np.log(ER.loc[t_exit, cur])

        # Record total period PnL on exit date
        portfolio_ret.loc[t_exit] += (-entry_cost - exit_cost)

    return portfolio_ret



# =====================================================
# 7. Compute hybrid momentum signals
# =====================================================

def compute_signals(ER, lookback=4, hold_period=22, pct=0.3,
                    ts_window=50, market_threshold=0):

    log_ret = compute_log_returns(ER)
    cs = compute_cross_sectional_momentum(log_ret, lookback)
    ts = compute_time_series_momentum(ER, ts_window)

    hybrid = combine_momentum(cs, ts)

    market_mom = compute_market_momentum(hybrid)
    regime = market_regime_multiplier(market_mom, market_threshold)

    hybrid_filtered = hybrid.mul(regime, axis=0)

    signals = build_signal_matrix(
        hybrid_filtered,
        ER.index,
        ER.columns,
        hold_period,
        pct
    )

    return signals, hybrid_filtered, log_ret



# =====================================================
# 8. Momentum using mid-only
# =====================================================

def momentum_ER_only(ER, signals, vol_window=60, target_vol=0.10):
    raw = compute_mid_execution(ER, signals)
    scale = compute_volatility_scaler(raw, vol_window, target_vol)
    return raw * scale



# =====================================================
# 9. Momentum using corrected bid/offer execution
# =====================================================

def momentum_bidask(BID, OFFER, ER, signals, hold_period=22,
                    vol_window=60, target_vol=0.10):

    raw = compute_bidask_execution(BID, OFFER, ER, signals, hold_period)
    scale = compute_volatility_scaler(raw, vol_window, target_vol)
    return raw * scale




# =====================================================
# 10. Wrapper
# =====================================================

def run_portfolio_strategies_improved(portfolios, lookback=4, hold_period=22, pct=0.3, print_data = True):

    results = {}

    for name, df_port in portfolios.items():

        BID, ER, OFFER = split_bid_offer(df_port)

        signals, momentum, log_ret_spot = compute_signals(
            ER,
            lookback=lookback,
            hold_period=hold_period,
            pct=pct
        )

        ret_mid = momentum_ER_only(ER, signals)
        stats_mid = evaluate_strategy(ret_mid, freq='D')

        ret_ba = momentum_bidask(BID, OFFER, ER, signals, hold_period)
        stats_ba = evaluate_strategy(ret_ba, freq='D')

        results[name] = {
            "ER_only": {
                "returns": ret_mid,
                "stats": stats_mid
            },
            "BidAsk": {
                "returns": ret_ba,
                "stats": stats_ba
            },
            "signals": signals,
            "momentum": momentum,
            "log_returns": log_ret_spot
        }
        if print_data: 
            print("\n==============================")
            print(f"Portfolio: {name}")
            print("=== ER-only ===")
            print(stats_mid)
            print("=== Bid/Ask ===")
            print(stats_ba)

    return results

portfolio_run_momentum_strat_biblio= run_portfolio_strategies_improved(portfolios)

summarize_portfolio(portfolio_run_momentum_strat_biblio, portfolios, "P1")
plot_all_portfolios_pnl(portfolio_run_momentum_strat_biblio)

In [ ]:
param_grid = [
    {"lookback": 4, "hold": 22, "pct": 0.2},
    {"lookback": 10, "hold": 22, "pct": 0.4},
]
# allow to test other setup -> we can maybe tune some values using optuna for bayesian optimisation of those parameters

subsamples = {
    "1999-2007": ("1999-01-01", "2007-12-31"), # (pre-crisis)
    "2008-2012": ("2008-01-01", "2012-12-31"), # (Financial Crisis + Euro Crisis)
    "2013-2019": ("2013-01-01", "2019-12-31"), # (QE, Low Volatility)
    "2020-2024": ("2020-01-01", "2024-12-31"), # (COVID, Inflation, QT)
}


results_sub = {}

for sub_name, (start, end) in subsamples.items():
    print(sub_name) 
    df_sub = xusd_df.loc[start:end]
    portfolios_sub = get_economic_portfolios(df_sub)
    if len(df_sub) < 200:
        continue

    for p in param_grid:
        lb = p["lookback"]
        hd = p["hold"]
        pct = p["pct"]
        portfolio_data = run_portfolio_strategies_improved(portfolios,lookback=lb, hold_period=hd, pct=pct, print_data = False)
        summarize_portfolio(portfolio_data, portfolios, "P1") #change the key to visualize the portfolio you wish to comment on 
        plot_all_portfolios_pnl(portfolio_data)

        key = f"{sub_name} | LB={lb}, HP={hd}, PCT={pct}"
        results_sub[key] = portfolio_data


Despite integrating state-of-the-art enhancements such as volatility scaling (Barroso & Santa-Clara, 2015), crash-robust conditioning (Daniel & Moskowitz, 2016) and hybrid trend specifications (Moskowitz et al., 2012), the FX momentum strategy does not exhibit any economically meaningful improvement. This is fully consistent with the empirical literature, which shows that momentum is structurally weak in FX spot markets, highly regime-dependent, and largely dominated by bid–ask frictions and global volatility conditions. In short, the strategy fails not because of implementation errors, but because FX momentum is not a robust standalone factor over long horizons.